# Feature Groups Notebook

When you onboard a dataset into Hybrid Intelligence, you can group related *numeric* features together, and these will be treated as a feature group within the ESM.

In this example, we will group the input features `CapitalGain` and `CapitalLoss` into a feature group called `Capital`. You can define feature groups when onboarding a dataset with the `feature_groups` parameter.

# Check Environment Variables
Before installing Hybrid Intelligence in the notebook you need to set these Environment Variables externally as described in the User Guide https://docs.umnai.com/set-up-your-environment. 
This section checks that the environment variables have been set correctly and throws an error if not.

In [1]:
import os

umnai_env_vars = {
    'UMNAI_CLIENT_ID',
    'UMNAI_CLIENT_SECRET',
    'PIP_EXTRA_INDEX_URL',
}

if not umnai_env_vars.issubset(os.environ.keys()):
    raise ValueError(
        'UMNAI environment variables not set correctly. They need to be set before using the Umnai library.'
    )

# Install Hybrid Intelligence
Next we install the UMNAI Platform.

In [2]:
%pip install umnai-platform --quiet


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


# Set Workspace Paths
Now we will set the the experiment name and workspace paths.

In [3]:
EXP_NAME = 'featuregroups_adult_income'
WS_PATH = 'resources/workspaces/'+EXP_NAME
EXP_PATH = EXP_NAME

# Import and Prepare Dataset
Import the dataset to a Pandas DataFrame and the clean data in preparation for onboarding into Hybrid Intelligence.

In [4]:
import pandas as pd
import numpy as np

# Import Adult Income Dataset to pandas dataframe: 
# This dataset can be downloaded from https://archive.ics.uci.edu/dataset/2/adult 
column_names = ["Age", "WorkClass", "fnlwgt", "Education", "EducationNum", "MaritalStatus", "Occupation", "Relationship", "Race", "Gender", "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]
dataset_df = pd.read_csv('https://raw.githubusercontent.com/umnaibase/umnai-examples/main/data/adult.data', names = column_names)

# Data Preparation:
dataset_df = dataset_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)    # Remove whitespaces
dataset_df["Income"] = np.where((dataset_df["Income"] == '<=50K'), 0, 1)                # Replace Target values with [0,1]
dataset_df.tail(5)

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,1


# Create or Open a Hybrid Intelligence Workspace
Workspaces are used by the Hybrid Intelligence framework to organize your data and models together in one place.

In [5]:
from umnai.workspaces.context import Workspace

# Open a workspace
ws = Workspace.open(
    path=WS_PATH,
    experiment=EXP_PATH
)

ws # Prints workspace details to confirm created/opened

# Onboard Hybrid Intelligence Dataset

Onboard the Pandas DataFrame into a Hybrid Intelligence dataset. 

You can specify feature groups by passing a dictionary where the key is the name of the group, and the values are the list of features to group together, to the `feature_groups` parameter.

In [6]:
from umnai.data.datasets import Dataset
from umnai.data.enums import PredictionType

dataset = Dataset.from_pandas(
    dataset_df,
    prediction_type=PredictionType.CLASSIFICATION,
    features=list(dataset_df.drop(['Income'], axis=1).columns),    # All columns except 'Income' are features
    targets=['Income'],
    feature_groups={'Capital': ['CapitalGain', 'CapitalLoss']}
)

dataset # Prints dataset details to confirm created/opened

MLFLOW Run ID: 197c8a79482f4b3a8962df87b475ba77:   0%|          | 0/36 [00:00<?, ?it/s]

Processing Tasks:   0%|          | 0/15 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Processing Tasks:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:tensorflow:Assets written to: /opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/featuregroups_adult_income/preprocessing/dataset_name=Dataset_1aa9572c/assets


Dataset(id=979a3c6c-3213-4f82-aded-7df48d3e5806; name=Dataset_1aa9572c; is_named=False; workspace_id=None)

# Confirm Feature Group Onboarding
When onboarding a dataset, all user defined feature groups are included in the dataset metadata.

In [7]:
dataset.feature_groups

{'Capital': ['CapitalGain', 'CapitalLoss']}

# Statistical Data
The statistical data for a feature group is still shown in terms of the individual input features.

In [8]:
pd.DataFrame(dataset.stats)

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
null_count,0.000000e+00,0.0,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.00000
minimum,1.700000e+01,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,1.000000e+00,NaN,0.00000
maximum,9.000000e+01,NaN,1.484705e+06,NaN,16.000000,NaN,NaN,NaN,NaN,NaN,9.999900e+04,4.356000e+03,9.900000e+01,NaN,1.00000
sum,1.256257e+06,NaN,6.179373e+09,NaN,328237.000000,NaN,NaN,NaN,NaN,NaN,3.508932e+07,2.842700e+06,1.316684e+06,NaN,7841.00000
unique_count,NaN,9.0,NaN,16.0,NaN,7.0,15.0,6.0,5.0,2.0,NaN,NaN,NaN,42.0,2.00000
mean,3.858165e+01,NaN,1.897784e+05,NaN,10.080679,NaN,NaN,NaN,NaN,NaN,1.077649e+03,8.730383e+01,4.043746e+01,NaN,0.24081
stddev,1.860557e+02,NaN,1.114046e+10,NaN,6.618687,NaN,NaN,NaN,NaN,NaN,5.454086e+07,1.623720e+05,1.524543e+02,NaN,0.18282


# Induce a Hybrid Intelligence Model

You do not need to make any alterations to the standard procedure to induce a model from a dataset that includes feature groups.

Pre-induced models are available in the notebook workspace on Github and may be downloaded and saved locally. Using Pre-induced models will speed up the execution of the notebook.

If `LOAD_PREINDUCED_MODEL` is set to `1` (default), the notebook will look for and load the pre-induced model with `ESM_ID`. Otherwise, if set to `0` or the pre-induced model is not found, a new model will be induced and saved to the workspace.

In [9]:
# Set this variable to '1' to load a pre-induced model, otherwise set to '0' to re-induce a new model from the dataset
LOAD_PREINDUCED_MODEL = 1

# Model ID
ESM_ID='Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb1c2'

#### Load or Induce the Model

In [10]:
from umnai.esm.model import ESM
from umnai.induction.inducer import ModelInducer

# Check if a saved model with the ESM_ID exists. If it exists load it, otherwise induce a new model, save it and print the model and run IDs
if (LOAD_PREINDUCED_MODEL == 1):
    try:    
        esm = ESM.from_workspace(id = ESM_ID)
        print('Pre-induced ESM loaded from workspace: ' + esm.id)
    except OSError:
        print("No model found in workspace.")
        LOAD_PREINDUCED_MODEL = 0

if (LOAD_PREINDUCED_MODEL == 0):
    print("Inducing a new model - this may take some time.")
    # Induce a simple model quickly using fast execution parameters
    model_inducer = ModelInducer(
        max_interactions=3,
        max_interaction_degree=2,
        max_polynomial_degree=2,
        trials=2,
        estimators=2,
        batch_size=512,
        iterations=2,
    )

    # # Induce a more realistic model using default Induction parameters:
    # model_inducer = ModelInducer()

    # Create an ESM using Induction
    esm = model_inducer.induce(dataset)

    # Save the ESM to your workspace
    esm.save_to_workspace()

    # Note ESM ID and MLFLow Run ID
    print("ESM ID: ", esm.id)
    print("MLflow Run ID: ", esm.producer_run_id)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Pre-induced ESM loaded from workspace: Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb1c2


# Inference: Query a Model 
When you query a Hybrid Intelligence model you get predictions together with explanations in real-time.

## Create a Query with Feature Groups

**No alterations are necessary** to create a query for a model with feature groups, the inputs are still applied to the original feature names.

In [11]:
from umnai.explanations.local import Query
import pandas as pd

query = Query({
    'Age': [39],
    'WorkClass': ['State-gov'],
    'fnlwgt': [77516],
    'Education': ['Bachelors'],
    'EducationNum': [13],
    'MaritalStatus': ['Never-married'],
    'Occupation': ['Adm-clerical'],
    'Relationship': ['Not-in-family'],
    'Race': ['White'],
    'Gender': ['Male'],
    'CapitalGain': [2174],
    'CapitalLoss': [0],
    'HoursPerWeek': [40],
    'NativeCountry': ['United-States']
})

### Instantiate a Local Explainer
Create a LocalExplainer to define the ESM you want to query. The local explainer lets you extract query explanations and predictions in real-time.

In [12]:
from umnai.explanations.local import LocalExplainer

# Instantiate a LocalExplainer:
local_explainer = LocalExplainer(esm)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


## Query Result with Feature Groups

When you are viewing a query result, the attributions will be in terms of the **feature group**.

### Submit the Query
Then you should pass the Query object to the local explainer instance. This will return a QueryResult that includes the prediction and the query explanation.

In [13]:
# Query the model:
query_result = local_explainer(query)

#  Display the Query Result together with the explanation
query_result.data

{'query_input': {'Age': array([39]),
  'WorkClass': array(['State-gov'], dtype=object),
  'fnlwgt': array([77516]),
  'Education': array(['Bachelors'], dtype=object),
  'EducationNum': array([13]),
  'MaritalStatus': array(['Never-married'], dtype=object),
  'Occupation': array(['Adm-clerical'], dtype=object),
  'Relationship': array(['Not-in-family'], dtype=object),
  'Race': array(['White'], dtype=object),
  'Gender': array(['Male'], dtype=object),
  'CapitalGain': array([2174]),
  'CapitalLoss': array([0]),
  'HoursPerWeek': array([40]),
  'NativeCountry': array(['United-States'], dtype=object)},
 'scenario_id': None,
 'context_id': None,
 'query_row_hash': array([153092249520382873164400753216439156323], dtype=object),
 'query_created_time': datetime.datetime(2024, 3, 1, 13, 29, 54, tzinfo=<UTC>),
 'model_id': 'Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb1c2',
 'model_intercept': -0.9549171,
 'dataset_id': '7c77bc1701764fe7857cc8892d2cd56b',
 'run_id': None,
 'run_created_time': 

# Explore and Explain a Model with Feature Groups

When you are exploring a model, the feature module and any interaction modules will be in terms of the **feature group**, while module rules will be in terms of the individual features in the group.

## ModelSummaryView
The Model Summary View gives you an overview of the key parameters, inputs and outputs of the model, and of each module within it.

In [14]:
from umnai.views.model_summary import ModelSummaryView

model_summary_view = ModelSummaryView(esm=esm)
model_summary_view.data

{'model_id': 'Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb1c2',
 'model_name': 'esm',
 'model_title': None,
 'model_created': datetime.datetime(2024, 1, 16, 15, 50, 34, tzinfo=<UTC>),
 'model_last_trained': datetime.datetime(2024, 1, 16, 15, 50, 34, tzinfo=<UTC>),
 'model_uvc': '3517d215633cbc1195fe6b1e6e9cbb334342e98fc0368860e3fc8b87f9f7f472',
 'model_intercept': -0.9549170732498169,
 'has_personal_individual_data': False,
 'has_reuse_restrictions': False,
 'model_doi': '',
 'model_copyright': '',
 'n_input_features': 14,
 'n_transformed_features': 108,
 'n_output_targets': 1,
 'features': ['Age',
  'WorkClass',
  'fnlwgt',
  'Education',
  'EducationNum',
  'MaritalStatus',
  'Occupation',
  'Relationship',
  'Race',
  'Gender',
  'CapitalGain',
  'CapitalLoss',
  'HoursPerWeek',
  'NativeCountry'],
 'targets': ['Income'],
 'n_modules': 13,
 'n_partitions': 22,
 'max_interaction_degree': 1,
 'model_interaction_count': 22,
 'max_width': 1,
 'max_depth': 3,
 'n_categorical_features':

## PartialDependencyView
The Partial Dependency View for the Feature Group module shows you the transfer function of the feature group components (input features) to the  module attribution.

In [15]:
from umnai.views.partial_dependency import PartialDependencyView

# Select Feature Group module
selected_module = 'Capital'

# Generate the view
partial_dependency_view = PartialDependencyView(esm=esm, module=selected_module, use_training_data=False)

# Display the results
partial_dependency_view.data

,input_feature.CapitalGain,input_feature.CapitalLoss,attribution.Income,attribution_normalized.Income,module_partition_index,rule_id,condition_expr_friendly,attribution_delta.Income
0,0,0,-0.144169,-0.000003,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,NaN
1,0,5,-0.142472,-0.000003,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.001697
2,0,17,-0.138398,-0.000003,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.004074
3,0,67,-0.121425,-0.000003,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.016973
4,0,70,-0.120407,-0.000003,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.001018
...,...,...,...,...,...,...,...,...
10297,99999,4296,9.672995,0.000201,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.006110
10298,99999,4318,9.680463,0.000201,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.007468
10299,99999,4341,9.688271,0.000201,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.007808
10300,99999,4342,9.688610,0.000201,0,55f1ee3782824fcb8181ab546c9e28ec,FOR ALL,0.000340


# Explain a Query with Feature Groups

When you create a Feature Attribution View or an Interaction Attribution View with a feature group, the attributions will appear in terms of the feature group. Additionally, you will see the input values of the feature group component features in the column called `grouped_features`. The expressions in the Interaction Attribution View will include terms that use the input features, rather than the feature group.

## FeatureAttributionView

In [16]:
from umnai.views.feature_attribution import FeatureAttributionView

# Create the view and display the data
feature_attribution_view = FeatureAttributionView(query_result)
feature_attribution_view.data

,input_feature,feature_attribution,feature_attribution_absolute,feature_attribution_normalized,grouped_features,feature_input
0,MaritalStatus,-0.729641,0.729641,0.353546,None,Never-married
1,Relationship,-0.399129,0.399129,0.193397,None,Not-in-family
2,Education,0.296275,0.296275,0.143559,None,Bachelors
3,EducationNum,0.179170,0.179170,0.086816,None,13
4,Occupation,-0.159792,0.159792,0.077427,None,Adm-clerical
5,Age,0.097970,0.097970,0.047471,None,39
6,fnlwgt,-0.073724,0.073724,0.035723,None,77516
7,Gender,0.059446,0.059446,0.028805,None,Male
8,Capital,0.037554,0.037554,0.018196,"[CapitalGain, CapitalLoss]","{'CapitalGain': 2174, 'CapitalLoss': 0}"
9,HoursPerWeek,-0.015076,0.015076,0.007305,None,40


## InteractionAttributionView

In [17]:
from umnai.views.interaction_attribution import InteractionAttributionView

# Create the view and display the data
interaction_attribution_view = InteractionAttributionView(query_result)
interaction_attribution_view.data

,module_id,module_index,module_name,module_partition_index,global_partition_index,rule_id,output_target_index,total_attribution,total_attribution_normalized,input_feature_0,grouped_features_0,feature_attribution_0,feature_input_0,condition_expr_friendly,summarized_then_expr
0,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,7,MaritalStatus,1,14,a86f7120d23a43539d6c680531bbfc54,0,-0.729641,0.353546,MaritalStatus,None,-0.729641,Never-married,"MaritalStatus = ""Never-married""",-0.729397535324097 - 0.000243463626247831*(Ma...
1,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,11,Relationship,0,19,8d2c808688244e2ca50b817aeb13335b,0,-0.399129,0.193397,Relationship,None,-0.399129,Not-in-family,"Relationship ≠ ""Husband""",-0.478880226612091 + 0.0797510817646980*(Rela...
2,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,5,Education,0,11,0eab85d53a9e4c9a8c66af7519e2ff3a,0,0.296275,0.143559,Education,None,0.296275,Bachelors,FOR ALL,-0.152660638093948 + 0.448935925960541*(Educa...
3,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,2,EducationNum,1,8,40a7ff5597ed4f59bcb06b74c75b3b3b,0,0.179170,0.086816,EducationNum,None,0.179170,13,EducationNum > 9.5,0.164189696311951 + 4.55372428405099e-5*Educa...
4,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,9,Occupation,0,17,4199b752f9994faeb4f1394423b01134,0,-0.159792,0.077427,Occupation,None,-0.159792,Adm-clerical,FOR ALL,-0.161185473203659 + 0.00139327999204397*(Occ...
5,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,0,Age,1,1,629450f0b1884d789bde59d7019038ab,0,0.097970,0.047471,Age,None,0.097970,39,(Age > 32.5) and (Age ≤ 43.5),0.0977788195014000 + 3.96826271753525e-6*Age ...
6,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,1,fnlwgt,0,3,e9a51028f11e46559cb7df8a252abd94,0,-0.073724,0.035723,fnlwgt,None,-0.073724,77516,fnlwgt ≤ 109863,-0.0737244188785553 + 9.69416907858900e-12*fn...
7,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,6,Gender,0,12,6ec377fbd2f04c75bba38614bbf86545,0,0.059446,0.028805,Gender,None,0.059446,Male,FOR ALL,-0.0973834991455078 + 0.156829938292503*(Gend...
8,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,4,Capital,0,10,55f1ee3782824fcb8181ab546c9e28ec,0,0.037554,0.018196,Capital,"[CapitalGain, CapitalLoss]",0.037554,"{'CapitalGain': 2174, 'CapitalLoss': 0}",FOR ALL,-0.144169241189957 + 8.35891332826577e-5*Capi...
9,Dataset_a1bbd0a0_18bd71a612b240d98ff1a2cc89adb...,3,HoursPerWeek,0,9,afdadd6ede8a4affa798cc8c76df1724,0,-0.015076,0.007305,HoursPerWeek,None,-0.015076,40,FOR ALL,-0.0890980362892151 + 1.03249130916083e-5*Hou...
